In [2]:
!pip install faiss-cpu sentence-transformers

In [5]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import json

model = SentenceTransformer("all-MiniLM-L6-v2")

# Load FAISS index and metadata for RAG
def load_faiss():
    global index, documents
    index = faiss.read_index("law_faiss.index")
    with open("law_metadata.json", "r", encoding="utf-8") as f:
        documents = json.load(f)

# Function to retrieve relevant sections
def retrieve_sections(query, top_k=3):
    query_embedding = model.encode(query).astype(np.float32)
    distances, indices = index.search(np.array([query_embedding]), top_k)

    results = []
    for i in range(top_k):
        if indices[0][i] != -1:
            section = documents[indices[0][i]]
            results.append({
                "law_type": section["law_type"],
                "chapter_number": section["chapter_number"],
                "chapter_title": section["chapter_title"],
                "content": section["content"],
                "source_url": section["source_url"]
            })
    return results

# Function to summarize sections
def summarize_sections(sections):
    summaries = []
    for section in sections:
        summary = f"{section['law_type']} Chapter {section['chapter_number']} - {section['chapter_title']}: {section['content'][:300]}..."
        summaries.append(summary)
    return "\n".join(summaries)

# Function to provide crime reporting steps
def report_steps(query):
    steps = {
        "theft": [
            "1. Visit the nearest police station.",
            "2. File an FIR (First Information Report) detailing the theft.",
            "3. Provide any evidence such as CCTV footage or witness statements.",
            "4. Obtain a copy of the FIR for reference.",
            "5. Follow up with the police regarding the investigation."
        ],
        "assault": [
            "1. Seek medical attention if needed.",
            "2. Report the incident at the nearest police station.",
            "3. File an FIR with detailed information about the assault.",
            "4. Provide evidence such as medical reports or witness statements.",
            "5. Follow up with the authorities for updates on the case."
        ]
    }
    for crime, step_list in steps.items():
        if crime in query.lower():
            return "\n".join(step_list)
    return "Please visit the nearest police station for guidance."

# Function to handle user query
def handle_query(query):
    sections = retrieve_sections(query)
    summary = summarize_sections(sections)
    steps = report_steps(query)

    response = f"Relevant Sections:\n{summary}\n\nSteps to Report:\n{steps}"
    return response

# Example usage
load_faiss()
query = "What are the laws regarding theft?"
print(handle_query(query))

Relevant Sections:
BNS Chapter 17 - Offences Against Property: Chapter
XVII
– Of Offences Against Property
Of Theft
Section
303
– Theft.
Whoever, intending to take dishonestly any movable property out of the possession of any person without that person’s consent, moves that property in order to such taking, is said to commit theft.
Explanation 1:
A thing so lon...
IPC Chapter 17 - Offences Against Property: Chapter
XVII
– Of Offences Against Property
Section
378
:- Theft
Whoever, intending to take dishonestly any movable property out of the possession of any person without that person’ consent, moves that property in order to such taking, is said to commit theft.
Explanations
A thing so long as it is a...
IPC Chapter 23 - Attempts to Commit Offences: Chapter
XXIII
– Of Attempts to Commit Offences
Section
511
:- Punishment for attempting to commit offences punishable with imprisonment for life or other imprisonment
Whoever attempts to commit an offence punishable by this Code with impri